https://tekore.readthedocs.io/en/stable/reference/client.html#playlist-api


* Spotify.playlist_change_details: Change a playlist's details.
* Spotify.playlists: Get a list of the playlists owned or followed by a user.
* Spotify.playlist_cover_image: Get cover image of a playlist.
* Spotify.playlist_cover_image_upload: Upload a custom playlist cover image.
* Spotify.playlist_items: Full details of items on a playlist.


In [17]:
import tekore as tk
conf = tk.config_from_environment()
token = tk.prompt_for_user_token(*conf)
spotify = tk.Spotify(token)


Opening browser for Spotify login...


In [ ]:
spotify.current_user()

In [ ]:
current_user = spotify.current_user().uri

In [ ]:
for i, playlist in enumerate(spotify.playlists('gingerale3').items):
    if playlist.owner.uri != spotify.current_user().uri:
        continue
    print(f"Idx {i}, name: {playlist.name}", )
    for j in playlist.images:
        print(f"height: {j.height}, width: {j.width}, url: {j.url}")

    print("---------------------------")

In [18]:



playlist = spotify.followed_playlists(limit=1).items[0]
track = spotify.playlist_items(playlist.id, limit=1).items[0].track
name = f'"{track.name}" from {playlist.name}'

if track.episode:
    print(f'Cannot analyse episodes!\nGot {name}.')
elif track.track and track.is_local:
    print(f'Cannot analyse local tracks!\nGot {name}.')
else:
    print(f'Analysing {name}...\n')
    analysis = spotify.track_audio_features(track.id)
    print(repr(analysis))

Analysing "Timeless Frequency" from Study / Sleep / Chill / Stream Lofi - DeadlyJimmy...

AudioFeatures with fields:
  acousticness = 0.956
  analysis_url = 'https://api.spotify.com/v1/audio-analysis/2Q7wCNeLOaO...'
  danceability = 0.713
  duration_ms = 124444
  energy = 0.101
  id = '2Q7wCNeLOaOmHvQTogMSy2'
  instrumentalness = 0.877
  key = 11
  liveness = 0.112
  loudness = -20.348
  mode = 1
  speechiness = 0.067
  tempo = 81.048
  time_signature = 4
  track_href = 'https://api.spotify.com/v1/tracks/2Q7wCNeLOaOmHvQTogMSy2'
  type = 'audio_features'
  uri = 'spotify:track:2Q7wCNeLOaOmHvQTogMSy2'
  valence = 0.119


In [19]:
import asyncio
import tekore as tk

from collections import Counter

def get_artist(track) -> str:
    if track.episode:
        return track.show.name
    else:
        return track.artists[0].name


In [20]:
async def count_artists(spotify: tk.Spotify, playlist_id: str):
    tracks = await spotify.playlist_items(playlist_id)
    tracks = spotify.all_items(tracks)
    return Counter([get_artist(t.track) async for t in tracks])


async def main() -> Counter:
    sender = tk.RetryingSender(sender=tk.AsyncSender())
    spotify = tk.Spotify(token, sender=sender, max_limits_on=True)

    playlists = await spotify.followed_playlists()
    playlists = spotify.all_items(playlists)
    counts = [await count_artists(spotify, p.id) async for p in playlists]

    await spotify.close()
    return sum(counts, Counter())


artists = asyncio.run(main())

for name, count in artists.most_common(3):
    print(count, name)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
%autoawait
result = await my_async_function()
